# Table of Contents
 <p><div class="lev1"><a href="#Measure-Dynamic-Functional-Connectivity"><span class="toc-item-num">1&nbsp;&nbsp;</span>Measure Dynamic Functional Connectivity</a></div><div class="lev2"><a href="#Initialize-Environment"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev2"><a href="#Gather-all-data-sets"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Gather all data sets</a></div><div class="lev2"><a href="#Generate-processing-list"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Generate processing list</a></div><div class="lev2"><a href="#Submit-proc_list-to-qsub-as-jobs"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Submit proc_list to qsub as jobs</a></div><div class="lev1"><a href="#Check-Adjacency"><span class="toc-item-num">2&nbsp;&nbsp;</span>Check Adjacency</a></div>

# Measure Dynamic Functional Connectivity

## Initialize Environment

In [1]:
try:
    %load_ext autoreload
    %autoreload 2
    %reset
except:
    print 'NOT IPYTHON'

from __future__ import division

import os
import sys
import glob

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import scipy.io as io
import h5py
import matplotlib.pyplot as plt
from matplotlib import rcParams

sys.path.append('/home/akhambhati/JagHome/hoth_research/Echobase')
import Echobase

rcParams = Echobase.Plotting.fig_format.update_rcparams(rcParams)

path_CoreData = '/data/jag/akhambhati/Remotes/CORE.RAM_Stim'
path_PeriphData = '/data/jag/akhambhati/Remotes/RSRCH.RAM_Stim'
path_ExpData = path_PeriphData + '/e01-FuncNetw'

for path in [path_CoreData, path_PeriphData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


/home/akhambhati/Developer/miniconda2/envs/echobase/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


## Gather all data sets

In [127]:
path_list = glob.glob('{}/Subjects/*.mat'.format(path_CoreData))

subj_dict = {}
for pth in path_list:    
    full_file = pth.split('/')[-1]    
    subj_id = full_file.split('.')[0]

    # Check if DataFile or DataFile_2
    channel_path = '{}/Subjects/{}.channel_list.mat'.format(path_CoreData, subj_id)
    if os.path.exists(channel_path):
        continue

    
    freq_id = full_file.split('.')[1]
    amp_id = full_file.split('.')[2]
    if not '_' in full_file.split('.')[3]:
        amp_id += '_{}'.format(full_file.split('.')[3])
    trial_id = full_file.split('.')[4]
    
    if not subj_id in subj_dict.keys():
        subj_dict[subj_id] = {}        
    if not freq_id in subj_dict[subj_id].keys():
        subj_dict[subj_id][freq_id] = {}
    if not amp_id in subj_dict[subj_id][freq_id].keys():
        subj_dict[subj_id][freq_id][amp_id] = {}
    if not trial_id in subj_dict[subj_id][freq_id][amp_id].keys():
        subj_dict[subj_id][freq_id][amp_id][trial_id] = {'raw_path': pth,
                                                         'raw_name': '{}.{}.{}.{}'.format(subj_id,
                                                                                          freq_id,
                                                                                          amp_id,
                                                                                          trial_id)}    

## Generate processing list

In [187]:
proc_item = []
for subj_id in subj_dict.keys():
    for freq_id in subj_dict[subj_id].keys():
        for amp_id in subj_dict[subj_id][freq_id].keys():
            for trial_id in subj_dict[subj_id][freq_id][amp_id].keys():

                raw_path = subj_dict[subj_id][freq_id][amp_id][trial_id]['raw_path']
                raw_name = subj_dict[subj_id][freq_id][amp_id][trial_id]['raw_name']
                res_path = '{}/Adjacency.{}.npz'.format(path_ExpData, raw_name)
                                
                if os.path.exists(res_path):
                    continue
                    
                proc_item.append({'raw_path': raw_path,
                                  'raw_name': raw_name,
                                  'res_path': res_path})
print('There are {} jobs to process.'.format(len(proc_item)))

There are 42994 jobs to process.


## Submit proc_list to qsub as jobs

In [188]:
# Submit proc_item list as jobs to qsub
job_file = open('./e01-Measure_Adjacency.job', 'w')
job_file.write('#!/bin/bash\n\n')
for pitem in proc_item:    
    stdout_path = '{}/Adjacency.{}.stdout'.format(path_ExpData, pitem['raw_name'])
    stderr_path = '{}/Adjacency.{}.stderr'.format(path_ExpData, pitem['raw_name'])
    
    py_str = './e01-Measure_Adjacency {} {}'.format(
        pitem['raw_path'], pitem['res_path'])

    cmd_str = 'qsub -cwd -o {} -e {} -l h_vmem=6.1G,s_vmem=6G'.format(stdout_path, stderr_path)

    job_file.write('{} {}\n'.format(cmd_str, py_str))
job_file.close()    

# Check Adjacency

In [ ]:
%matplotlib inline

adj = np.load('{}/Adjacency.R1084T.Freq_50.Amp_1_5.Trial_370.npz'.format(path_ExpData))['adj'].item()

print('Pre-Stimulus Adjacency')
plt.figure()
ax = plt.subplot(2,2,1); ax.matshow(adj['Pre_Stim']['AlphaTheta'], cmap='viridis', vmin=0, vmax=1); ax.set_axis_off();
ax = plt.subplot(2,2,2); ax.matshow(adj['Pre_Stim']['Beta'], cmap='viridis', vmin=0, vmax=1); ax.set_axis_off();
ax = plt.subplot(2,2,3); ax.matshow(adj['Pre_Stim']['LowGamma'], cmap='viridis', vmin=0, vmax=1); ax.set_axis_off();
ax = plt.subplot(2,2,4); ax.matshow(adj['Pre_Stim']['HighGamma'], cmap='viridis', vmin=0, vmax=1); ax.set_axis_off();
plt.show()

print('Post-Stimulus Adjacency')
plt.figure()
ax = plt.subplot(2,2,1); ax.matshow(adj['Post_Stim']['AlphaTheta'], cmap='viridis', vmin=0, vmax=1); ax.set_axis_off();
ax = plt.subplot(2,2,2); ax.matshow(adj['Post_Stim']['Beta'], cmap='viridis', vmin=0, vmax=1); ax.set_axis_off();
ax = plt.subplot(2,2,3); ax.matshow(adj['Post_Stim']['LowGamma'], cmap='viridis', vmin=0, vmax=1); ax.set_axis_off();
ax = plt.subplot(2,2,4); ax.matshow(adj['Post_Stim']['HighGamma'], cmap='viridis', vmin=0, vmax=1); ax.set_axis_off();
plt.show()

print('Delta Adjacency')
plt.figure()
ax = plt.subplot(2,2,1); ax.matshow(adj['Post_Stim']['AlphaTheta']-adj['Pre_Stim']['AlphaTheta'],
                                    cmap='viridis', vmin=-1, vmax=1); ax.set_axis_off();
ax = plt.subplot(2,2,2); ax.matshow(adj['Post_Stim']['Beta']-adj['Pre_Stim']['Beta'],
                                    cmap='viridis', vmin=-1, vmax=1); ax.set_axis_off();
ax = plt.subplot(2,2,3); ax.matshow(adj['Post_Stim']['LowGamma']-adj['Pre_Stim']['LowGamma'],
                                    cmap='viridis', vmin=-1, vmax=1); ax.set_axis_off();
ax = plt.subplot(2,2,4); ax.matshow(adj['Post_Stim']['HighGamma']-adj['Pre_Stim']['HighGamma'],
                                    cmap='viridis', vmin=-1, vmax=1); ax.set_axis_off();
plt.show()

Pre-Stimulus Adjacency
